In [23]:
bucket = 'hy2568'
prefix = 'newHope'
import sagemaker
role = sagemaker.get_execution_role()

In [24]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-2 region. You will use the 825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [25]:
bucket_name = 'hy2568' # <--- change this variable to a unique name for your bucket
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [46]:
train_data = pd.read_csv('./new_train_data.csv',index_col=0)
test_data = pd.read_csv('./new_test_data.csv',index_col=0)
train_data


,NumberOfReviews,Recommended,"Cuisine_['Asian Fusion', 'Chinese', 'Indian']","Cuisine_['Chinese', 'Indian', 'Breakfast & Brunch']","Cuisine_['Delis', 'Indian', 'Pakistani']","Cuisine_['Food Trucks', 'Indian']","Cuisine_['Gastropubs', 'Indian', 'Breakfast & Brunch']","Cuisine_['Halal', 'Pakistani', 'Indian']","Cuisine_['Indian', 'Bangladeshi', 'Halal']","Cuisine_['Indian', 'Bangladeshi']",...,"Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Mediterranean']","Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Specialty Food']","Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Turkish']","Cuisine_['Wine Bars', 'Tapas/Small Plates']","Cuisine_['Wine Bars', 'Turkish']","Cuisine_['Wine Bars', 'Tuscan']","Cuisine_['Wine Bars', 'Venues & Event Spaces', 'American (New)']","Cuisine_['Wine Bars', 'Wine Tasting Classes']",Cuisine_['Wine Bars'],Cuisine_[]
Rating,,,,,,,,,,,,,,,,,,,,,
4.5,1331,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.5,391,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.5,473,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,667,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.5,364,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.5,228,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.5,869,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.5,273,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,522,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
test_data

,NumberOfReviews,Recommended,"Cuisine_['Asian Fusion', 'Chinese', 'Indian']","Cuisine_['Chinese', 'Indian', 'Breakfast & Brunch']","Cuisine_['Delis', 'Indian', 'Pakistani']","Cuisine_['Food Trucks', 'Indian']","Cuisine_['Gastropubs', 'Indian', 'Breakfast & Brunch']","Cuisine_['Halal', 'Pakistani', 'Indian']","Cuisine_['Indian', 'Bangladeshi', 'Halal']","Cuisine_['Indian', 'Bangladeshi']",...,"Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Mediterranean']","Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Specialty Food']","Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Turkish']","Cuisine_['Wine Bars', 'Tapas/Small Plates']","Cuisine_['Wine Bars', 'Turkish']","Cuisine_['Wine Bars', 'Tuscan']","Cuisine_['Wine Bars', 'Venues & Event Spaces', 'American (New)']","Cuisine_['Wine Bars', 'Wine Tasting Classes']",Cuisine_['Wine Bars'],Cuisine_[]
Rating,,,,,,,,,,,,,,,,,,,,,
4.0,73,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.5,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,68,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.5,87,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,63,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3.5,37,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.5,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
pd.concat([train_data['Recommended'], train_data.drop(['Recommended'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
s3_input_train

In [30]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],role, train_instance_count=1, train_instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [31]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2019-04-03-00-45-49-095


2019-04-03 00:45:49 Starting - Starting the training job...
2019-04-03 00:45:50 Starting - Launching requested ML instances......
2019-04-03 00:46:53 Starting - Preparing the instances for training...
2019-04-03 00:47:49 Downloading - Downloading input data
2019-04-03 00:47:49 Training - Downloading the training image...
2019-04-03 00:48:15 Uploading - Uploading generated training model
Arguments: train
[2019-04-03:00:48:10:INFO] Running standalone xgboost training.
[2019-04-03:00:48:10:INFO] Path /opt/ml/input/data/validation does not exist!
[2019-04-03:00:48:10:INFO] File size need to be processed in the node: 1.04mb. Available memory size in the node: 8389.96mb
[2019-04-03:00:48:10:INFO] Determined delimiter of CSV input is ','
[00:48:10] S3DistributionType set as FullyReplicated
[00:48:10] 200x2732 matrix with 546400 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[00:48:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 4


2019-04-03 00:48:23 Completed - Training job completed
Billable seconds: 48


In [38]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2019-04-03-01-40-49-773
INFO:sagemaker:Creating endpoint with name xgboost-2019-04-03-00-45-49-095


---------------------------------------

KeyboardInterrupt: 

In [39]:
test_data

,NumberOfReviews,Recommended,"Cuisine_['Asian Fusion', 'Chinese', 'Indian']","Cuisine_['Chinese', 'Indian', 'Breakfast & Brunch']","Cuisine_['Delis', 'Indian', 'Pakistani']","Cuisine_['Food Trucks', 'Indian']","Cuisine_['Gastropubs', 'Indian', 'Breakfast & Brunch']","Cuisine_['Halal', 'Pakistani', 'Indian']","Cuisine_['Indian', 'Bangladeshi', 'Halal']","Cuisine_['Indian', 'Bangladeshi']",...,"Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Mediterranean']","Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Specialty Food']","Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Turkish']","Cuisine_['Wine Bars', 'Tapas/Small Plates']","Cuisine_['Wine Bars', 'Turkish']","Cuisine_['Wine Bars', 'Tuscan']","Cuisine_['Wine Bars', 'Venues & Event Spaces', 'American (New)']","Cuisine_['Wine Bars', 'Wine Tasting Classes']",Cuisine_['Wine Bars'],Cuisine_[]
Rating,,,,,,,,,,,,,,,,,,,,,
4.0,73,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.5,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,68,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.5,87,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,63,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3.5,37,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.5,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
test_data_array = test_data.drop(['Recommended'], axis=1).as_matrix() #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
a1 = test_data_array[0:501]
a2 = test_data_array[501:1001]
a3 = test_data_array[1001:1501]
a4 = test_data_array[1501:2001]
a5 = test_data_array[2001:2501]
a6 = test_data_array[2501:3001]
a7 = test_data_array[3001:3501]
a8 = test_data_array[3501:4001]
a9 = test_data_array[4001:4501]
a10 = test_data_array[4501:5001]
a11 = test_data_array[5001:5501]
a12 = test_data_array[5501:]
test = [a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12]

In [86]:
result = []
for list in test:
    predictions = xgb_predictor.predict(list).decode('utf-8') # predict!
    predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
    for i in range(0,len(list)):
        if predictions_array[i] > 0.6:
            predictions_array[i] = 1
        else:
            predictions_array[i] = 0
#         predictions_array[i] = int(round(predictions_array[i]))
    result.extend(predictions_array)

In [87]:
len(result)

5922

In [88]:
result

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0

In [93]:
test_data = pd.read_csv('./new_test_data.csv',index_col=0)
test_data['Recommended'] = result

In [96]:
test_data

,NumberOfReviews,Recommended,"Cuisine_['Asian Fusion', 'Chinese', 'Indian']","Cuisine_['Chinese', 'Indian', 'Breakfast & Brunch']","Cuisine_['Delis', 'Indian', 'Pakistani']","Cuisine_['Food Trucks', 'Indian']","Cuisine_['Gastropubs', 'Indian', 'Breakfast & Brunch']","Cuisine_['Halal', 'Pakistani', 'Indian']","Cuisine_['Indian', 'Bangladeshi', 'Halal']","Cuisine_['Indian', 'Bangladeshi']",...,"Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Mediterranean']","Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Specialty Food']","Cuisine_['Wine Bars', 'Tapas/Small Plates', 'Turkish']","Cuisine_['Wine Bars', 'Tapas/Small Plates']","Cuisine_['Wine Bars', 'Turkish']","Cuisine_['Wine Bars', 'Tuscan']","Cuisine_['Wine Bars', 'Venues & Event Spaces', 'American (New)']","Cuisine_['Wine Bars', 'Wine Tasting Classes']",Cuisine_['Wine Bars'],Cuisine_[]
Rating,,,,,,,,,,,,,,,,,,,,,
4.0,73,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.5,26,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,68,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,6,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.5,87,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.0,1,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,63,0.0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3.5,37,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.5,8,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
test_data.to_csv('./prediction_result.csv',columns=test_data.columns,index=True,sep=',')